In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union, unary_union, transform
from datetime import datetime, timedelta
import datetime
import math
from scipy.ndimage.interpolation import shift
import shapely.wkt
from shapely.validation import explain_validity,make_valid
import xarray as xr
import pygeos as pg

import seaborn as sns

In [21]:
#fire_daily = gpd.read_file('lake_VIIRS_daily.geojson') #polygons and attributes
fire_daily = gpd.read_file('august_complex_VIIRS_daily.geojson') #polygons and attributes

fire_daily=fire_daily.drop(columns=['Current Overpass'])
fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])

fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares
fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
fire_daily

,Current Day,Incident Number,Fire Name,Lat Fire,Lon Fire,Number of NEW VIIRS points,NEW FRP,geometry,fire area (ha)
0,2020-08-18,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,44.0,1885.55,"MULTIPOLYGON (((3537745.042 1018363.527, 35377...",1135.344377
1,2020-08-19,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,1248.0,61233.26,"MULTIPOLYGON (((3540508.047 1020032.346, 35404...",29696.858369
2,2020-08-20,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,1164.0,54976.00,"MULTIPOLYGON (((3538666.643 1013813.846, 35386...",20847.906367
3,2020-08-21,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,799.0,13657.05,"MULTIPOLYGON (((3534338.100 1012918.362, 35343...",10260.069207
4,2020-08-22,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,761.0,8077.88,"MULTIPOLYGON (((3534620.688 1015522.235, 35346...",6052.243045
5,2020-08-23,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,729.0,11105.64,"MULTIPOLYGON (((3525776.295 1016033.983, 35257...",6013.719580
6,2020-08-24,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,213.0,10146.56,"MULTIPOLYGON (((3537621.422 1008843.957, 35376...",2014.407508
7,2020-08-25,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,630.0,8314.40,"MULTIPOLYGON (((3527034.918 1012910.637, 35270...",4595.631819
8,2020-08-26,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,469.0,3679.33,"MULTIPOLYGON (((3533508.555 1014371.403, 35335...",3482.791629
9,2020-08-27,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,430.0,6953.32,"MULTIPOLYGON (((3523710.437 1016868.353, 35237...",3467.561180


In [3]:
merra_grid = gpd.read_file('MERRA_GRID.geojson')
merra_grid = merra_grid.to_crs(epsg=3347) #put into lambert conformal conic

In [22]:
fire_merra_intersection = gpd.overlay(fire_daily, merra_grid, how='intersection',keep_geom_type=False)
fire_merra_intersection['grid intersection area (ha)'] =fire_merra_intersection['geometry'].area/10000
fire_merra_intersection['weights'] = fire_merra_intersection['grid intersection area (ha)']/fire_merra_intersection['fire area (ha)'] 
#fire_merra_intersection.set_index('Current Day')
fire_merra_intersection

,Current Day,Incident Number,Fire Name,Lat Fire,Lon Fire,Number of NEW VIIRS points,NEW FRP,fire area (ha),lat,lon,geometry,grid intersection area (ha),weights
0,2020-08-18,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,44.0,1885.55,1135.344377,39.0,-123.750,"MULTIPOLYGON (((3525971.213 1031029.681, 35259...",61.910254,0.054530
1,2020-08-19,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,1248.0,61233.26,29696.858369,39.0,-123.750,"MULTIPOLYGON (((3527728.093 1030938.882, 35277...",926.631521,0.031203
2,2020-08-20,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,1164.0,54976.00,20847.906367,39.0,-123.750,"MULTIPOLYGON (((3531874.068 1038777.273, 35318...",1560.434459,0.074848
3,2020-08-21,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,799.0,13657.05,10260.069207,39.0,-123.750,"MULTIPOLYGON (((3532931.897 1040777.206, 35329...",432.788388,0.042182
4,2020-08-22,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,761.0,8077.88,6052.243045,39.0,-123.750,"MULTIPOLYGON (((3532992.198 1041031.667, 35329...",758.025782,0.125247
5,2020-08-23,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,729.0,11105.64,6013.719580,39.0,-123.750,"MULTIPOLYGON (((3532954.350 1040993.930, 35329...",1276.887807,0.212329
6,2020-08-24,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,213.0,10146.56,2014.407508,39.0,-123.750,"MULTIPOLYGON (((3532142.222 1040514.348, 35321...",526.543704,0.261389
7,2020-08-25,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,630.0,8314.40,4595.631819,39.0,-123.750,"MULTIPOLYGON (((3532276.652 1039538.399, 35322...",1512.334200,0.329081
8,2020-08-26,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,469.0,3679.33,3482.791629,39.0,-123.750,"MULTIPOLYGON (((3532291.391 1039808.205, 35322...",1529.134995,0.439054
9,2020-08-27,11843929.0,AUGUST COMPLEX,39.659444,-122.808889,430.0,6953.32,3467.561180,39.0,-123.750,"MULTIPOLYGON (((3532289.677 1039800.430, 35322...",1244.275245,0.358833


In [23]:
fire_merra_intersection = fire_merra_intersection.set_index(['Current Day', 'lat', 'lon'])

In [24]:
dat_intersection = fire_merra_intersection.to_xarray()
print(dat_intersection)

<xarray.Dataset>
Dimensions:                      (Current Day: 71, lat: 3, lon: 3)
Coordinates:
  * Current Day                  (Current Day) object '2020-08-18' ... '2020-...
  * lat                          (lat) float64 39.0 39.5 40.0
  * lon                          (lon) float64 -124.4 -123.8 -123.1
Data variables:
    Incident Number              (Current Day, lat, lon) float64 nan ... nan
    Fire Name                    (Current Day, lat, lon) object nan ... nan
    Lat Fire                     (Current Day, lat, lon) float64 nan ... nan
    Lon Fire                     (Current Day, lat, lon) float64 nan ... nan
    Number of NEW VIIRS points   (Current Day, lat, lon) float64 nan ... nan
    NEW FRP                      (Current Day, lat, lon) float64 nan ... nan
    fire area (ha)               (Current Day, lat, lon) float64 nan ... nan
    geometry                     (Current Day, lat, lon) object nan ... nan
    grid intersection area (ha)  (Current Day, lat, lon) float

In [25]:
path_merra = '/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.202008*.nc4'
#              '/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20200815.nc4']
dat_merra = xr.open_mfdataset(path_merra,concat_dim='time',combine='nested',compat='override', coords='all')
print(dat_merra)

<xarray.Dataset>
Dimensions:   (lat: 37, lon: 39, time: 744)
Coordinates:
  * lat       (lat) float64 31.0 31.5 32.0 32.5 33.0 ... 47.5 48.0 48.5 49.0
  * lon       (lon) float64 -125.0 -124.4 -123.8 -123.1 ... -102.5 -101.9 -101.2
  * time      (time) datetime64[ns] 2020-08-01 ... 2020-08-31T23:00:00
Data variables:
    PS        (time, lat, lon) float32 dask.array<chunksize=(24, 37, 39), meta=np.ndarray>
    QLML      (time, lat, lon) float32 dask.array<chunksize=(24, 37, 39), meta=np.ndarray>
    SPEEDLML  (time, lat, lon) float32 dask.array<chunksize=(24, 37, 39), meta=np.ndarray>
    TLML      (time, lat, lon) float32 dask.array<chunksize=(24, 37, 39), meta=np.ndarray>
Attributes: (12/31)
    History:                           Wed Jul 14 21:21:06 2021: ncks -d lon,...
    Comment:                           GMAO filename: d5124_m2_jan10.inst1_2d...
    Filename:                          MERRA2_400.inst1_2d_lfo_Nx.20200801.nc4
    Conventions:                       CF-1
    Institut

In [27]:
merra_daily_mean=dat_merra.resample(time='D').mean(dim='time')

merra_aug14 = merra_daily_mean.sel(lat = np.unique(dat_intersection['lat'].values),
                                  lon = np.unique(dat_intersection['lon'].values))
                                  #time = np.array(dat_intersection['Current Day'].values, dtype='datetime64'))
print(merra_aug14)

<xarray.Dataset>
Dimensions:   (lat: 3, lon: 3, time: 31)
Coordinates:
  * time      (time) datetime64[ns] 2020-08-01 2020-08-02 ... 2020-08-31
  * lat       (lat) float64 39.0 39.5 40.0
  * lon       (lon) float64 -124.4 -123.8 -123.1
Data variables:
    PS        (time, lat, lon) float32 dask.array<chunksize=(1, 3, 3), meta=np.ndarray>
    QLML      (time, lat, lon) float32 dask.array<chunksize=(1, 3, 3), meta=np.ndarray>
    SPEEDLML  (time, lat, lon) float32 dask.array<chunksize=(1, 3, 3), meta=np.ndarray>
    TLML      (time, lat, lon) float32 dask.array<chunksize=(1, 3, 3), meta=np.ndarray>


In [39]:
print(np.nansum(merra_aug14['PS'].values[0:10,:,:]*dat_intersection['weights'].values[0:10,:,:],axis=(1,2)))

[95649.90649188 95714.75364821 94819.81289533 95689.07457765
 96100.99815445 96203.48297001 96635.65610927 96525.02716181
 97545.35044995 96797.67351583]


In [ ]:
times_end = np.minimum(np.datetime64(fire_merra_intersection['Current Day'].iloc[len(fire_merra_intersection)-1])+np.timedelta64(1,'D'), 
                      np.datetime64('2020-10-31')+np.timedelta64(1,'D'))
times = np.arange(np.datetime64(fire_merra_intersection['Current Day'].iloc[0]),\
                         times_end)
print(times)

ind = 0
for ii in range(len(times)):
    #print(times[ii])
    if str(times[ii]) in fire_merra_intersection['Current Day'].values:
        print(fire_merra_intersection.iloc[ind])
        ind=ind+1
    else:
        print('doot doot')
    

In [ ]:
np.datetime64(fire_merra_intersection['Current Day'].values)

In [ ]:
plot_twoline(fire_merra_intersection['Current Day'], np.stack([fire_merra_intersection['NEW FRP'], fire_merra_intersection['fire area (ha)']]),'LAKE',\
             [], ['VPD (hpa)', 'CO (kg)'], 1)

In [ ]:
#plots two lines, on one or on the same axis
# IN: datax, x axis vector (time)
# IN: datay, 2*n vector of the data
# IN: linelabels, string vector of the labels of each line, to go in the legend
# IN: axis labels, labels of the axis
# IN: title, string for title of figire
# IN: twoaxis, if 1 plot data on two separate axes, else plot on the same axis
# OUT: a graph
def plot_twoline(datax, datay, title, linelabels, axislabels, twoaxis):
    fig, ax = plt.subplots(figsize=(20,6)) #set up the figure
    
    if twoaxis==1:
        ax.plot(datax, datay[0,:], 'ro-') # plot the first line
        ax.set_ylabel(axislabels[0], fontsize=24, color='red') 
        ax2=ax.twinx()
        ax2.plot(datax, datay[1,:], 'bo-') #plot the second line
        ax2.set_ylabel(axislabels[1], fontsize=24, color='blue')
        
    else:
        ax.plot(datax, datay[0,:], 'ro-') # plot the first line
        ax.plot(datax, datay[1,:], 'bo-') # plot the second line
        ax.set_ylabel(axislabels, fontsize=24) 
    #print(np.arange(1,len(datax.values),6))
    plt.xticks(np.arange(1,len(datax.values),6))    
    plt.legend(linelabels,fontsize=24)
    plt.title(title,fontsize=24)
    plt.grid()
    plt.show()

In [ ]:
print(fire_merra_intersection)

fire_merra_intersection['Current Day']= pd.to_datetime(fire_merra_intersection['Current Day'])

fire_merra_intersection = fire_merra_intersection.set_index('Current Day')
print(fire_merra_intersection)
print(fire_merra_intersection.resample('D', convention='end').asfreq())